# Day 3

## Where we're headed

You're going to add on two new features to your code today. These features have the following things in common:
 - they're logically more difficult to implement
 - their behavior is dependent on how exactly you're running Python. That is, mac vs windows, stand alone executable vs Jupyter notebook, and what drivers etc you have installed

These two features are: the ability to gather data until the user stops acquisition, and actually acquiring a voltage off a DAQ board instead of the dummy random numbers we've been doing. Along the way we'll switch to an executable script, so that you can see what that looks like.

**Before you go further**, you need to decide how you're going to run the DAQ board: on your computer or on a lab computer. There is a note on Blackboard with information about what additional drivers you need to download. If you decide you need to install the National Instruments driver on your computer, you might want to get that started now in the background while you work. It's a disgustingly bulky set of programs that can take an hour or more to download and install.

## Allowing the user to interrupt data acquisiton

It's experimentally annoying to have to know how long your trial will take before it's done. It's a lot easier to just press go on the computer, acquire data for a while, and then stop it when you're done. So we're gonna change the logic of your `while` loop to make this happen.

### A bit about infinite loops and keyboard interrupts

An infinite loop is a loop that will never exit. The simplest way to write one is like this:

```while True
   do aThing
   do anotherThing
```

This loop will run forever, because the logic condition of the `while` loop never changes. You may have generated a inifinte loop at some point in this experiment by forgetting to make your iterator count up, or by getting the logic of your time variables mixed up. Infinite loops are a rite of passage for new coders.

You may have come across the way out of an infinte loop, which is to press `CTRL+C` or `CMD-C`. This is called a keyboard interrupt, and stops the current executation. This shortcut is useful beyond infinite loops. For example, perhaps your code has a bug that makes it eat up memory, or it's just gonna take hours to finish the task you accomplished. Maybe you it `CTRL+C` or `CMD-C` to kill the execution and try it again another way.

(You may also have noticed that it doesn't work in a Jupyter notebook. The keyboard command for a keyboard interrupt is `i i`, which has to do with the way web browsers process keyboard inputs. Also, you have to be in what is called 'command mode', which means the active cell has a blue edge. To get that, you can press `esc` or just click out of the cell.)

I've written an infinite loop below. Run it, and use the correct keyboard interrput to kill it.

In [ ]:
import time

i = 1

while True:
    print(i)
    i = i+1
    time.sleep(.5)


### Making a keyboard interrupt work for us

This way of exiting an infinite loop is actually really useful, if we just clean it up a bit. The loop I gave you does execute until the user stops it, and you can put anything you want into that infinite loop. The only problem with this version is that it raises an exception and stops at the keyboard interrupt, and we want it to wrap up the acquisition and save the data. To do that, we're gonna write in a `try-except` statement to handle the keyboard interrupt more smoothly.

You may have noticed that when your programs fail certain words crop up over and over again. For instance, you may have seen `ParseError`, `ValueError`, `TypeError`, or `NameError` show up. These are examples of the baseline ways in which a program can fail. Python has a set of built in errors (exceptions), and every time a program fails it tells you how it failed. (You can actually write your own error types and have Python use your particular error code, which is super useful in advanced applications.) `KeyboardInterrupt` is one of these exceptions, so we can work with it.

A `try-except` statement is a way of handling exceptions cleanly. It allows you to plan for problems. For instance, maybe a certain calculation in your script could result in dividing by zero. That would raise an exception and stop execution of your program. But if you know it's a possibility, maybe you want your program to work around this possiblity and keep going, instead of stopping where the error occurs. That's where exception handling can really shine. For more on exceptions, here's the Python tutorial page on errors:
https://docs.python.org/3/tutorial/errors.html

The structure of a `try-except` statement goes like this:
```
try:
    aThing() #here is where your loop will end up
except ExceptionName:
    print('alright moving on now') #it's nice to put a note that says we made it to this point in the program
    
print('this line only happens after the exception is raised.')
#this is probably where you'll end up putting the lines where you'll save your data
```

In that script, `aThing()` would be executed *unless* `ExceptionName` had been raised, then it would print out `alright moving on now` and go on with whatever lines are next.

Your task is to combine the infinite loop I've written above with a `try-except` statement. Write a bit of code that prints out numbers forever, up until the user presses a keyboard interrupt, then it should move on and do something else. Don't try to do everything your script does just yet. Get the logic working first.

### Adding in your script

Alright now that you have the `try-except` statement working on simple code, combine it with the script you had at the end of day two. You'll want to wrap your existing loop in the `try-except` syntax and change the exit condition of your while loop. You should end up with a program that does the following things:
 - Asks the user to define the data acquisition rate
 - Acquires data (random numbers) at that rate until the user hits the keyboard interrupt
 - Plots the data
 - Saves it just like it did before

For now, it's okay if you're only plotting after the fact, but leave yourself a sign that the program is acquiring. Maybe it prints out the most recently acquired random number each cycle, or something like that. The final version of your program will need to plot real time, but that'll be less annoying once you move over to Spyder.

Clean up your code. Are there lines not being use? Probably. Also, put a note somewhere telling the user how to get out of your loop. You might put this as a printed prompt, or as the title to your plot.

# Switching to Spyder

Alrigh it's time to make the big move! You've been copying and pasting yoru script into different Jupyter notebooks for three weeks, time to give it it's own home. I'm going to write the following for folks using Anaconda. If you're using something else, you probably know how to do this. If not, gimme a shout in class.

Go to your start menu, find the Anaconda folder, and click on Spyder. If it's not there, open up Anaconda Navigator and open Spyder from there.

Spyder is what's called an integrated development environment, or IDE. It's somewhere between hard core "I'm a Linux user and can debate VIM versus Emacs and know 17 programming languages" and Jupyter notebooks. While the hard core option isn't bad, it's probably unnecessary for all of you. However, it's nice to know that IDEs aren't actually necessary. I wrote my thesis editing the code in Notepad and running it from the terminal. That's a totally feasible way to work, and if you like it, go ahead and do it.

If you've used MATLAB before, you'll notice that Spyder looks a lot like MATLAB. There's a pane with your script, a help window, and a console window.

Take the currently working version of your script and past it into the script window in Spyder. Edit the top comment to show your name and what your script does. Save it with a simple filename ("dataAcquire.py"). Then press the run button to verify that it works. Remember that exiting your inifinte loop will be a different key combo in Spyder.

If your plots are appearing in the console window, you need to change a setting. Go to Tools->Preferences->IPython console->Graphics->Graphics backend and select "Automatic" instead of "inline". You may need to restart Spyder for this to take effect. It'll cause plots to open in a separate window instead of putting them in the console. That window will probably appear *behind* Spyder.


## Acquiring a voltage from the DAQ board

### Importing the library

The last functionality you need to add to your script is the little tiny detail of actually acquiring data instead of using random numbers. To do this, you'll need to be on a computer with the National Instruments drivers, and you'll need to plug in a DAQ board. See the note on Blackboard.

Firstly, you'll need to add the following line to your script (think about where it goes):

'import nidaqmx'

Next, I'm gonna tell you a lot of information without having you write any code. Settle in to read for a bit.

### Getting started with DAQ boards

A DAQ board is a little device that takes a raw voltage and gives it to the computer. DAQ stands for Data AQuisition. They're also called things like ADC (for analog-to-digital-conversion) or I/O (input/output) devices. As the last name suggests, you can usually output a voltage to a DAQ board also. Our DAQ boards are manufactured by National Instruments. You can see an image of our DAQ boards here:
https://www.ni.com/en-us/support/model.usb-6009.html

You'll notice that it has two rows pins along the two edges. If you look at one in person, they're all labeled things like 'GND' and 'AI3'. What we want is the ability to read in the voltage output of the thermocouple circuit relative to the circuit ground. Since we're not externally grounding our circuit, we want to take the difference between those two points, rather than referencing the output voltage to some external ground. This is a really common circumstance, so the DAQ boards are set up to make that easy. If you use two adjacent analog inputs, the board reads the difference between them. For our purposes, we'll plug the output voltage of the thermocouple circuit into 'AI0' and the ground into 'AI4' right next to it.

To connect your circuit, you use a screwdriver to back off the screw, insert the wire, and them screw it down. However, if nothing is plugged in, the board will happily read zero volts off that channel. It's easier to troubleshoot one thing at a time, so don't connect your circuit right now. Just leave it empty (or ask Dr. V how to plug in a 9V battery if you want something to read).

### How we talk to a DAQ board

In order to talk to a DAQ board, you have to know what the computer has called it, and then pick what channel you want to talk to. It's possible that you'd have more than one DAQ board each reading several voltages, so the software is set up to let you keep all those straight.

You can see what your DAQ board is called by opening up the "NI MAX" program that got installed when you downloaded all of that stuff from National Instruments. Under "Devices and Interfaces" you'll see something called "NI USB-6009 Dev1" or similar. Take note of what device number it is.

The idea is first you create a channel to the particular voltage you want to read, which means you know what input it's on and the device name. Then you can manually read voltages off that channel. When you're done, you need to close the channel so that it's ready to open up again the next time you want to read it.

### Let's actually do something!

The three lines of code below open up a channel to analog input 0 of device 3 and read one voltage. Convince yourself that you're seeing functions that seem to say that. Edit it to put in the correct device name for your computer. Then execute it and see what output you get. What voltage should you get? Plug in a 9V battery and see if you get what you expect.

A note: Put in the wrong device number. Notice that it raises a special exception: `DaqError`. Look at the bottom of the error. It suggests the correct device name. It's often faster and easier to do this instead of opening up the NI MAX program to check what device number you're using.

A note: You might get a scary pink error box giving a ResourceWarning and saying that your device was not properly closed. This is true. More to come.

In [ ]:
task=nidaqmx.Task()
task.ai_channels.add_ai_voltage_chan('Dev3/ai0')
task.read(number_of_samples_per_channel=1)

What we're doing is manually opening the channel and reading the voltage. But notice that we don't close the channel. That's where the scary pink box comes from. 

For operations where Python has to work with something like a file or a voltage channel, the proper thing to do is open your thing, work with it, then close it. You might be saying, wait, we didn't have to manually open or close the file when writing our data, but that's because `np.savetxt()` has the open and close steps built in, to make sure their handled correctly. The reason you have to work open-write-close, rather than just open-write, is that the file being written is still in the memory. If you try to write to that same file again, really strange behavior can result.

Alright so we want to do open-write-close on files, and we want to do the same with voltage channels for the same reason. **But** what if something bad happens while you're working with the file or voltage channel, and causes your script to stop? The file or voltage channel never gets closed. It's still in the memory as being used, and when you try to run again you can get strange behavior (or scary pink boxes. or both). Enter the `with` statement.


A `with` statement is designed to *automatically* handle the closing part, even if the script ends unexpectedly. They work like this:

```
with thing.to.do() as nickname:
    nickname.openOrWhatever()
    doSomething()
```

Your version will look something like this:
```
with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan('Dev4/ai0')
    doSomething()
```

The advantage to working with a `with` statement is that if the script fails in the `doSomething()` section, the DAQ board still gets closed down properly for it's next usage.

Use the space below to write a script that opens up a DAQ board with a `with` statement, adds on a channel, and reads and prints ten voltages from the DAQ board.

## Finish it up!

 Alright it's time to put all of this together. Take a deep breath, and make the following magic happen over in Spyder.
 
You're going to wrap the `try-except` statement into a `with` statment. It'll structured like the code below. You'll then want to comment out the call to a random number, and instead store the voltage as your data point.


In [ ]:
#your imports go here
#user inputs go here
#initializing arrays and whatnot go here

with athing() as thing:
    thing.open()
    try:
        while True:
            all your loop stuff
    except KeyboardInterrupt:
        print('ending data acquisition')

#the lines to combine your arrays and save them go here

# Exit ticket

The grading rubric for this assignment is on Blackboard. As a group, read it, and decide what grade you're getting. You are allowed to leave when your group decides that you're happy with your grade, or at 5pm, whichever comes first. If you don't finish today, you'll need to tell Dr. V your plan for finishing it at home.

A note: if you're working on it at home, you won't have a DAQ board. You'll want to switch back to random numbers on your y-axis to get your script to execute. But be sure the version you turn in calls the DAQ board for full points.